### Duplikate
Es gibt keine Duplikate in Zeilen und keine in Reihen.

### Nicht relevante Daten:
- Languages
- Netflix Link	
- IMDb Link	
- Summary
- Image
- Poster
- TMDb Trailer	
- Trailer Site
- Tags
- Country Availability
- Director
- Writer
- Actors

Da diese String Werte sind und schwer bzw. gar nicht encoded werden können

### Ausreißer
Bei Rotten Tomatoes score gibt es 0 Werte.
Sonst gibt keine Ausreißer, aber manche Filme wurden im Vergleich zu anderen sehr oft bewertet.

### Vorverarbeitungsschritte
- Series or Movie kann Ordinal Encoded werden
- View Rating kann Ordinal Encoded werden
- Genre kann One-Hot Encoded werden
- Runtime kann zu <30 min -> 1, 1-2h -> 3 encoded werden
- Release Date kann zu Unix Date umgewandelt werden
- Boxoffice kann von String zu Number convertiert werden
- NaN Werte in Awards Nominated For kann auf 0 gesetzt werden.




In [16]:
import pandas as pd

# Read the CSV file into a DataFrame
file_path = './archive/netflix-rotten-tomatoes-metacritic-imdb.csv'
df = pd.read_csv(file_path)

pd.set_option('display.max_columns', None)

In [17]:
df['Awards Nominated For'] = df['Awards Nominated For'].fillna(0)

In [ ]:
# Select only string columns
string_columns = df.select_dtypes(include=[object])

# Print the string columns
string_columns

In [ ]:
columns_to_drop = ['Languages', 'Netflix Link', 'IMDb Link', 'Summary', 'Image', 'Poster', 'TMDb Trailer', 'Trailer Site', 'Tags', 'Country Availability', 'Production House','Director','Writer','Actors']
df = df.drop(columns=columns_to_drop)
df

In [ ]:
# Encode 'Runtime' column
df['Runtime'] = df['Runtime'].replace({'< 30 minutes': 1, '30-60 mins': 2, '1-2 hour': 3, '> 2 hours': 5, '> 2 hrs': 5})

df['Runtime']

In [ ]:
# Define the mapping dictionary for view ratings to minimum age
df['View Rating'] = df['View Rating'].fillna('Unrated')
view_rating_to_age = {
    'G': 0,
    'E10+': 10,
    'TV-Y7-FV': 7,
    'PG': 10,
    'PG-13': 13,
    'R': 17,
    'NC-17': 17,
    'MA-17': 17,
    'Approved': 0,
    'Passed': 0,
    'Unrated': 0,
    'UNRATED': 0,
    'E': 0,
    'Not Rated': 0,
    'NOTRATED': 0,
    'NOT RATED': 0,
    'TV-Y': 0,
    'TV-Y7': 7,
    'TV-G': 0,
    'AL': 0,
    'GP': 0,
    'TV-PG': 10,
    'TV-14': 14,
    'M/PG': 15,
    '15': 15,
    'U': 0,
    'M': 15,
    'TV-MA': 17,
    'X': 18,
    'TV-13': 13,
}

# Convert 'View Rating' to minimum age
df['Minimum Age'] = df['View Rating'].replace(view_rating_to_age)
df = df.drop('View Rating', axis=1)

# Display the count of each rating
df['Minimum Age'].value_counts()

In [ ]:
# Remove non-numeric characters and convert to float
df['Boxoffice'] = df['Boxoffice'].replace('[\$,]', '', regex=True).astype(float)

# Display the 'Boxoffice' column to verify the changes
df['Boxoffice']

In [ ]:
# Convert 'Series or Movie' to 0 for movies and 1 for series
df['Series or Movie'] = df['Series or Movie'].replace({'Movie': 0, 'Series': 1})
# Rename the column to 'Is Series'
df.rename(columns={'Series or Movie': 'Is Series'}, inplace=True)
# Display the updated column to verify the changes

df['Is Series']

In [ ]:
x = df[df['Release Date'].isna() & df['Netflix Release Date'].isna()]
nan_counts = df['Release Date'].isna().sum()
print(x)
print(nan_counts)

In [ ]:

df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')
df['Release Date'] = pd.to_datetime(df['Netflix Release Date'], errors='coerce', dayfirst=False)

df['Release Date'] = df['Release Date'].fillna(df['Netflix Release Date'])
df['Release Date']

In [ ]:
import numpy as np

# Print all rows where 'Release Date' is NaN and 'Netflix Release Date' is not



# Convert 'Release Date' to Unix timestamp (int) in seconds, only for non-NaT values
df['Release Date'] = df['Release Date'].where(df['Release Date'].isna(), df['Release Date'].view('int64') // 10**9)

# Replace NaT with np.nan (for consistency)
df['Release Date'] = df['Release Date'].where(df['Release Date'].notna(), np.nan)

# Print the count of NaN values in each column
df = df.drop('Netflix Release Date', axis=1)

In [ ]:
# Print all columns of all rows with NaN values
# Print percentage of NaN values in each column
nan_percentage = df.isna().mean() * 100
nan_percentage = nan_percentage.to_frame(name='Percentage of NaN Values')
pd.set_option('display.max_rows', None)

nan_percentage


In [28]:
pd.set_option('display.max_rows', 10)


In [ ]:
# Remove rows with any NaN values
df = df.dropna()

df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select only numeric columns
numeric_data = df.select_dtypes(include=[float, int])

# Create a correlation matrix
corr_matrix = numeric_data.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()


In [ ]:


genre_one_hot = df['Genre'].str.split(',').apply(lambda x: [item.strip() for item in x])
onehot_encoded = genre_one_hot.explode().str.get_dummies().groupby(level=0).sum()
df = pd.concat([df, onehot_encoded], axis=1)

df = df.drop('Genre', axis=1)

onehot_encoded

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the numeric data
scaled_numeric_data = scaler.fit_transform(numeric_data)

# Convert the scaled data back to a DataFrame
scaled_numeric_data = pd.DataFrame(scaled_numeric_data, columns=numeric_data.columns)

scaled_numeric_data

In [ ]:

# Create a horizontal boxplot for the numeric data
sns.boxplot(data=scaled_numeric_data, orient='h')
plt.title('Horizontal Boxplot of Numeric Data')
plt.show()

In [ ]:


# Create a horizontal boxplot for the numeric data
sns.boxplot(data=numeric_data, orient='h')
plt.title('Horizontal Boxplot of Numeric Data')
plt.show()


In [ ]:

votes = np.log1p(numeric_data)
sns.boxplot(data=votes, orient='h')
plt.title('Horizontal Boxplot of Numeric Data')
plt.show()

In [ ]:
# Select only string columns
string_columns = df.select_dtypes(include=[object])

# Print the string columns
string_columns

In [37]:
df.to_csv('preped.csv', index=False)

# The End

Continue [regression](./ML/tree.ipynb)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Select the features and target variable
features = df.select_dtypes(include=[float, int]).drop(columns=['IMDb Score'])
# features = df[['Is Series', 'Hidden Gem Score', 'Runtime', 'Rotten Tomatoes Score', 'Metacritic Score', 'Awards Received', 'Awards Nominated For', 'Boxoffice', 'Release Date', 'IMDb Votes', 'Minimum Age']]
target = df['IMDb Score']

# Drop rows with missing values in features or target
features = features.dropna()
target = target[features.index]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Scatter plot of actual vs. predicted IMDb scores
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual IMDb Score')
plt.ylabel('Predicted IMDb Score')
plt.title('Actual vs. Predicted IMDb Scores')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

import matplotlib.pyplot as plt

# Round the predicted and actual IMDb scores to the nearest integer
y_pred_rounded = np.round(y_pred)
y_test_rounded = np.round(y_test)

# Create the confusion matrix
conf_matrix = confusion_matrix(y_test_rounded, y_pred_rounded)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test_rounded), yticklabels=np.unique(y_test_rounded))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

# Select the features and target variable
features = df[['Hidden Gem Score', 'Runtime', 'Rotten Tomatoes Score', 'Metacritic Score', 'Awards Received', 'Boxoffice', 'IMDb Votes']]
target = df['IMDb Score']

# Drop rows with missing values in features or target
features = features.dropna()
target = target[features.index]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Scatter plot of actual vs. predicted IMDb scores
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual IMDb Score')
plt.ylabel('Predicted IMDb Score')
plt.title('Actual vs. Predicted IMDb Scores')
plt.show()

In [42]:
# from sklearn.tree import DecisionTreeRegressor

# # Create and train the decision tree regressor
# tree_model = DecisionTreeRegressor(random_state=42)
# tree_model.fit(X_train, y_train)

# # Make predictions on the test set
# y_tree_pred = tree_model.predict(X_test)

# # Evaluate the decision tree model
# tree_mse = mean_squared_error(y_test, y_tree_pred)
# tree_r2 = r2_score(y_test, y_tree_pred)

# print(f'Decision Tree Mean Squared Error: {tree_mse}')
# print(f'Decision Tree R-squared: {tree_r2}')

In [43]:
# from sklearn.tree import plot_tree
# import dtreeviz

# # Set the size of the plot
# plt.figure(figsize=(20, 10))

# # Plot the decision tree with a maximum depth of 3
# # Visualize the decision tree with dtreeviz
# viz = dtreeviz.model(tree_model, X_train, y_train,
#                target_name='IMDb Score',
#                feature_names=features.columns)

# # Display the visualization
# viz.view()
# plt.show()